## **_Backtesting Template_**

**_A step by step walkthrough of the backtesting process. Shouldn't be touched._**

## **_Libraries_**

In [2]:
# Backtesting
import vectorbt as vbt

# Data
import yfinance as yf
import pandas as pd
import numpy as np

# Time
import datetime as dt

# Plotting
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## **_Data_**

### **_Docstring:_**

**_"Title" Double tag_**

**_"Description:" Triple tag. Brief description of function._**

**_"Args:" Triple tag. name, type, description_**

**_"Returns:" Triple tag. name, type, description_**

### **_Code:_**

**_Variables in lowercase_**

**_Only returns series of close data_**

In [3]:
def Data(symbols, interval, start, end):
    """
    ## Data

    ### Description:
    This function collects close data from Yahoo Finance
    in a structure that agrees with the VectorBT Ind
    Factory function.

    ### Args:
    - symbols (List): List of symbol's data to collect
    - period (Integer): Number of most recent days to data to collect
    - interval (String): Timeframe of data to collect

    ### Returns:
    - price (Dataframe): VectorBT agreaeble dataframe of price data
    """

    # Downloading close data
    price = vbt.YFData.download(symbols=symbols,
                                missing_index="drop",
                                start=start,
                                end=end,
                                interval=interval).get("Close")
    
    # Returning the close prices
    return price

## **_Custom Indicator_**

### **_Docstring:_**

**_"Title" Double tag_**

**_"Description:" Triple tag. Brief description of function._**

**_"Args:" Triple tag. name, type, description_**

**_"Returns:" Triple tag. name, type, description_**

### **_Code:_**

**_Variables in lowercase_**

**_Outer for loop: Iterates through columns to handle multiple symbols_**

**_Inner for loop: Iterates through values in each column_**

In [4]:
def Custom_Indicator(price, ma_window, std_size):
    """
    ## Custom Indicator

    ### Description:
    A basic template of a BBands indicator.

    ### Args:
    - price (Dataframe): Pandas dataframe of close data
    - ma_window (Int): An integer for the moving average window
    - std_size (Int): An integer for the coefficient of the standard deviation

    ### Returns:
    - signals (Array): Numpy array of integer signals representing quantities to buy
    """

    # Initialising a dataframe of empty signal inputs
    signals = pd.DataFrame(index=price.index, columns=price.columns)

    # Looping through each column
    for column in price.columns:

        # Calculating the moving average and standard deviation
        moving_avg = price[column].rolling(window=ma_window).mean()
        standard_dev = price[column].rolling(window=ma_window).std()

        # Calculating the bands
        upper_band = moving_avg + (std_size * standard_dev)
        lower_band = moving_avg - (std_size * standard_dev)

        # Looping through each value of each column
        for idx, val in enumerate(price[column]):
            if val > upper_band.iloc[idx]:
                signals.at[idx, column] = -1
            elif val < lower_band.iloc[idx]:
                signals.at[idx, column] = 1
            else:
                signals.at[idx, column] = 0
    
    # Converting the signals to an array
    return np.array(signals.dropna())

## **_Custom Backtest_**

### **_Docstring:_**

**_"Title" Double tag_**

**_"Description:" Triple tag. Brief description of function._**

**_"Args:" Triple tag. name, type, description_**

**_"Returns:" Triple tag. name, type, description_**

### **_Code:_**

**_Variables in lowercase_**

**_"Finding Signals": Vars must match those in vbt ind. Param-product=True_**

**_"Calculating performance": size must have .astype(int)_**

In [5]:
def Custom_Backtest(price, ma_window, std_size, vbt_ind, cash):
    """
    ## Custom Backtest

    ### Description:
    A basic template to backtest a strategy

    ### Args:
    - start (Datetime): Start of backtesting period
    - end (Datetime): End of backtesting period
    - timeframe (String): Interval / timeframe of strategy
    - symbols (List): List of strings of symbols
    - vbt_ind (Vbt Object): VectorBT object wrapping your custom indicator
    - ma (List): List of moving average values
    - std (List): List of standard deviation values
    - cash (Int): An integer for starting cash

    ### Returns:
    - None
    """

    # Finding signals
    signals = vbt_ind.run(Price=price,
                          Moving_Average_Win=ma_window,
                          Standard_Deviation_Mag=std_size,
                          param_product=True)
    
    # Calcualting performance
    portfolio = vbt.Portfolio.from_orders(price,
                                          init_cash=cash,
                                          size=signals.Output.astype(int),
                                          size_type="Amount",
                                          freq="D")
    
    return signals, portfolio

## **_Custom VBT Indicator_**

### **_Docstring:_**

**_"Title" Double tag_**

**_"Description:" Triple tag. Brief description of function._**

**_"Args:" Triple tag. name, type, description_**

**_"Returns:" Triple tag. name, type, description_**

### **_Code:_**

**_"Class name": Reference to strategy_**

**_"Short name": Short reference to strategy_**

**_"Input names": Dataframe of close data for multiple symbols_**

**_"Param names": Reference to names of params, used in "Finding Signals"_**

**_"Output names": Reference to signal output of indicators_**

**_.from-apply-func: Reference to base indicator and defaults_**

In [6]:
Custom_Vbt_Indicator = vbt.IndicatorFactory(
        class_name="BBands Strategy",                                 
        short_name="BBands",                                          
        input_names=["Price"],                                        
        param_names=["Moving_Average_Win", "Standard_Deviation_Mag"], 
        output_names=["Output"]                                       
        ).from_apply_func(Custom_Indicator,                           
                          Moving_Average_Win=21,                      
                          Standard_Deviation_Mag=1,                   
                          keep_pd=True)                               

## **_Custom Plotting_**

### **_Docstring:_**

**_"Title" Double tag_**

**_"Description:" Triple tag. Brief description of function._**

**_"Args:" Triple tag. name, type, description_**

**_"Returns:" Triple tag. name, type, description_**

### **_Code:_**

**_Indicators: Recalculating indicators for respective strategy_**

**_Trades: Gathering trades from portfolio chosen backtest set_**

In [7]:
def Custom_Plotting(signals, portfolio, set):
    """
    ##  Custom Plotting

    ### Decription:
    A function to plot the trades of a specific backtest and position status throughout.
    Also plots indicators of chosen strategy.

    ### Args:
    - portfolio (VectorBT Portfolio object): Portfolio of specific backtest
    - signals (Object): Object of various dataframes of data and signals derived from said data
    - set (Tuple): A tuple of parameters of the backtest you want to plot

    ### Returns:
    - None
    """

    # Calculating the indicators
    data = pd.DataFrame()
    data["Price"] = signals.Price[set]
    data["Moving Avg"] = data["Price"].rolling(set[0]).mean()
    data["Standard Dev"] = data["Price"].rolling(set[0]).std()
    data["Upper Band"] = data["Moving Avg"] + (set[1] * data["Standard Dev"])
    data["Lower Band"] = data["Moving Avg"] - (set[1] * data["Standard Dev"])

    # Gathering trades
    trades = portfolio.orders.records_readable[portfolio.orders.records_readable["Column"]==set]
    trades["Colour"] = trades["Side"].apply(lambda side: "green" if side == "Buy" else "red")
    trades["Marker"] = trades["Side"].apply(lambda side: "triangle-up" if side == "Buy" else "triangle-down")

    # Calculating the position 
    pos_count_current = 0
    pos_count = []
    for i in range(len(signals.Output[set])):
        pos_count_current += signals.Output[set].iloc[i]
        pos_count.append(pos_count_current)
    pos_count = [f"Position: {pos}" for pos in pos_count]

    # Creating the figure
    fig = go.Figure()

    # Plotting the close price
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Price"],
                             mode="lines",
                             name="Price",
                             hovertext=pos_count))
    
    # Plotting the moving average
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Moving Avg"],
                             mode="lines",
                             name="Moving Average"))
    
    # Plotting the upper band
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Upper Band"],
                             mode="lines",
                             name="Upper Band"))
    
    # Plotting the lower band
    fig.add_trace(go.Scatter(x=data.index,
                             y=data["Lower Band"],
                             mode="lines",
                             name="Lower Band"))
    
    # Plotting the trades
    fig.add_trace(go.Scatter(x=trades["Timestamp"],
                             y=trades["Price"],
                             mode="markers",
                             marker=dict(color=trades["Colour"], symbol=trades["Marker"], size=10),
                             name="Buys + Sells"))
    
    # Updating the layout
    fig.update_layout(title=dict(text=f"Backtest: {set}, Profit: ${portfolio.total_profit()[set]}", font=dict(color="white")),
                      height=600,
                      paper_bgcolor="rgba(70,70,70,1)",
                      plot_bgcolor="rgba(230,230,230,1)",
                      xaxis=dict(title="Date",
                                 tickfont=dict(color="white"),
                                 titlefont=dict(color="white"),
                                 gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                      yaxis=dict(title="Price",
                                 tickfont=dict(color="rgba(230,230,230,1)"),
                                 titlefont=dict(color="rgba(230,230,230,1)"),
                                 gridcolor="rgba(0,0,0,0.1)", gridwidth=2),
                      shapes = [go.layout.Shape(type="rect",
                                                xref="paper",
                                                yref="paper",
                                                x0=0,
                                                y0=0,
                                                x1=1,
                                                y1=1,
                                                line={'width': 3, 'color': 'black'})],
                      legend=dict(font=dict(color="rgba(230,230,230,1)")))
    
    # Displaying the figure
    fig.show()

## **_Workspace_**

### **_Data:_**

**_Variable amounts of symbols_**

**_Datetime objects to determine start and end_**

### **_Backtest:_**

**_Parameters to optimise must be in a list_**

**_Reference the previously created VectorBT indicator_**

### **_Plotting:_**

**_Include signals and portfolio objects from backtest_**

**_Tuple of set parameters and symbol of backtest you wish to plot_**

In [10]:
# Gathering data
price = Data(["AAPL", "AMZN", "PLTR"], "1m", dt.datetime(year=2024, month=2, day=8), dt.datetime(year=2024, month=2, day=9))

# Backtesting
signals, portfolio = Custom_Backtest(price, [21, 22, 23, 24, 25, 26, 27, 28, 35, 42, 49], [1, 1.5, 2, 2.5, 3], Custom_Vbt_Indicator, 1000)

# Plotting
Custom_Plotting(signals, portfolio, portfolio.total_return().idxmax())